# Spotify Song Suggestor
-Natural Language Processing Model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from re import compile as rcompile
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [35]:
from google.colab import files
file = files.upload()

Saving spotify_songs.csv to spotify_songs (1).csv


In [3]:
#dataset used https://www.kaggle.com/imuhammad/audio-features-and-lyrics-of-spotify-songs
df = pd.read_csv(r"/Users/Carl/Desktop/datascience/spotify_song_suggester/data/spotify_songs.csv")

In [4]:
df.shape

(18454, 25)

In [5]:
df.isnull().sum()

track_id                      0
track_name                    0
track_artist                  0
lyrics                      260
track_popularity              0
track_album_id                0
track_album_name              0
track_album_release_date      0
playlist_name                 0
playlist_id                   0
playlist_genre                0
playlist_subgenre             0
danceability                  0
energy                        0
key                           0
loudness                      0
mode                          0
speechiness                   0
acousticness                  0
instrumentalness              0
liveness                      0
valence                       0
tempo                         0
duration_ms                   0
language                    260
dtype: int64

In [6]:
df.tail()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
18449,7zXzWCVuz7whIuYZyEAlxt,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,1l4aoukbPgi5u2OaE2R4Zj,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,...,-5.778,0,0.0878,0.00555,0.000000,0.335,0.211,128.012,208656,en
18450,7zycSpvjDcqh6YT1FEl2kY,Anaconda,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon...",49,5qs8T6ZHSrnllnOuUk6muC,The Pinkprint (Deluxe Edition),2014-12-15,10er Playlist,1kEczIkZH8IgaWT2BiApxZ,...,-6.224,1,0.1800,0.06730,0.000006,0.214,0.647,129.990,260240,en
18451,7zye9v6B785eFWEFYs13C2,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,1xdgLmTFMSyJyI5DJOOX7T,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,5EMARioe9z9eKOeWIAC2JW,...,-6.457,0,0.0270,0.71500,0.000428,0.115,0.657,142.218,191205,en
18452,7zyLObYw4QUKQDyZOb4J0Y,I'll Do 4 U (Re-Recorded / Remastered),Father MC,(Would you do for me) Sweetheart (Would you do...,36,14HYMxFhpgDIr9cci1u0kt,I'll Do 4 U (Re-Recorded / Remastered),2010-10-01,New Jack Swing/ R&B Hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,...,-4.920,0,0.0633,0.14300,0.000000,0.072,0.810,109.536,223890,en
18453,7zzZmpw8L66ZPjH1M6qmOs,Migraine,Moonstar88,"Oo nga pala, hindi nga pala tayo Hanggang dito...",61,4t3FtECyV1gClHmpBhXSfB,When I Met You,2008-07-28,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-6.000,1,0.0290,0.21700,0.000003,0.118,0.221,115.049,267960,tl


In [40]:
def wrangle(data):
    #drop null values
    data = data.dropna()
    
    #restrict to songs with lyrics in English
    data = data[data['language']=='en']
    
    #drop duplicates
    data = data.drop_duplicates(subset = ['track_name', 'track_artist'],
                                             keep = 'first')
    
    #reset index
    data = data.reset_index()
    
    return data

In [41]:
df2 = wrangle(df)

In [42]:
df2.shape

(13726, 26)

In [43]:
df2.isnull().sum()

index                       0
track_id                    0
track_name                  0
track_artist                0
lyrics                      0
track_popularity            0
track_album_id              0
track_album_name            0
track_album_release_date    0
playlist_name               0
playlist_id                 0
playlist_genre              0
playlist_subgenre           0
danceability                0
energy                      0
key                         0
loudness                    0
mode                        0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
duration_ms                 0
language                    0
dtype: int64

In [44]:
regx = rcompile('[^a-zA-Z 0-9]')

tokenize = lambda x: regx.sub('', x.lower().replace(',', ' ').replace('-',' '))

df2['lyrics'] = df2.lyrics.apply(tokenize)

In [45]:
display(df2.lyrics)

0        the trees  are singing in the wind the sky blu...
1        na yeah  spyderman and freeze in full effect u...
2        i really cant stay baby its cold outside ive g...
3        get up out of my business you dont keep me fro...
4        hold your breath  dont look down  keep trying ...
                               ...                        
13721    i had to catch the bus alarm was shaking  oh a...
13722    yeah  nah  nah  nah  nah  nah  nah  nah  nah n...
13723    caught up in such a head rush  wide eyed latel...
13724    bound  bound bound  bound bound to fall in lov...
13725    would you do for me sweetheart would you do fo...
Name: lyrics, Length: 13726, dtype: object

In [46]:
df2.head()

,index,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,the trees are singing in the wind the sky blu...,28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,rock,hard rock,0.303,0.880,9,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
1,2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,na yeah spyderman and freeze in full effect u...,0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,r&b,new jack swing,0.845,0.652,6,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
2,3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,i really cant stay baby its cold outside ive g...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,r&b,neo soul,0.425,0.378,5,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
3,4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,get up out of my business you dont keep me fro...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,pop,dance pop,0.760,0.887,9,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en
4,5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,hold your breath dont look down keep trying ...,70,3GNzXsFbzdwM0WKCZtgeNP,Chapters,2019-04-26,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,r&b,urban contemporary,0.496,0.639,6,-6.157,1,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720,en


In [47]:
#creating feature matrix and target vector
target = 'track_id'
y = df2[target]
X = df2.drop(target, axis=1)

In [48]:
X_train, y_train, X_test, y_test = train_test_split(X, y)

In [49]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10294, 25), (3432, 25), (10294,), (3432,))

In [50]:
vect = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2), min_df=0.03, 
                        max_df=0.25) 

# Create a vocabulary and get word counts per document 
dtm = vect.fit_transform(X_train) 

# Get feature names to use as dataframe column headers 
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names()) 
print(dtm.shape) 
dtm.head()

(25, 25)


,acousticness,danceability,duration_ms,energy,index,instrumentalness,key,language,liveness,loudness,lyrics,mode,playlist_genre,playlist_id,playlist_name,playlist_subgenre,speechiness,tempo,track_album_id,track_album_name,track_album_release_date,track_artist,track_name,track_popularity,valence
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [51]:
dtm_filename = 'nlp_dtm.pkl' 
pickle.dump(dtm, open(dtm_filename, 'wb'))

In [52]:
dtm_filename = 'nlp_dtm.pkl'

dtm = pickle.load(open(dtm_filename, 'rb'))

In [53]:
nn = NearestNeighbors(n_neighbors=6, algorithm='kd_tree') 
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [54]:
model_filename = 'nlp_model.pkl' 
pickle.dump(nn, open(model_filename, 'wb'))

In [55]:
def select_nearest_songs(id):

    track_num = df.loc[df['track_id']==id]
    x = track_num.index
    x = x[0]
    x = x.item()

    doc = dtm.loc[x].values
    result = loaded_model.kneighbors([doc], n_neighbors=6)
    
    rec_songs = {"id":[]};
    
    for i in range(5):
        song = result[1][0][1+i]

        id = df.loc[song]['track_id']
        
        rec_songs['id'].append(id)
    
    return rec_songs

In [56]:
select_nearest_songs('00cqd6ZsSkLZqGMlQCR0Zo')

{'id': ['00f9VGHfQhAHMCQ2bSjg3D',
  '00chLpzhgVjxs1zKC9UScL',
  '0017A6SJgTbfQVU2EtsPNo',
  '004s3t0ONYlzxII9PLgU6z',
  '00emjlCv9azBN0fzuuyLqy']}

In [57]:
select_nearest_songs('004s3t0ONYlzxII9PLgU6z')

{'id': ['00cqd6ZsSkLZqGMlQCR0Zo',
  '00chLpzhgVjxs1zKC9UScL',
  '0017A6SJgTbfQVU2EtsPNo',
  '00f9VGHfQhAHMCQ2bSjg3D',
  '00emjlCv9azBN0fzuuyLqy']}

In [96]:
print(f'numpy=={np.__version__}')
print(f'pandas=={pd.__version__}')
#print(f'scikit-learn=={sklearn.__version__}')

numpy==1.19.5
pandas==1.1.5
